In [1]:
# 1. Creating database file for analysis:
#    Forming table for nodes based on nodes.csv file
    
import sqlite3
import csv
from pprint import pprint

sqlite_file = "project.db"
conn = sqlite3.connect(sqlite_file)
conn.text_factory = str
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS nodes')
conn.commit()

cur.execute('''
    Create Table nodes(id INTEGER, lat REAL, lon REAL, user TEXT, uid INTEGER, version TEXT, changeset INTEGER, timestamp TEXT)
''')

conn.commit()


with open('nodes.csv','r') as file:
    table = csv.DictReader(file)
    to_table = [(i['id'], i['lat'], i['lon'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in table]
    cur.executemany("INSERT INTO nodes(id, lat, lon, user, uid, version, changeset, timestamp) VALUES (?,?,?,?,?,?,?,?);", to_table)

conn.commit()

In [2]:
# 2. Getting number of nodes in the table:

cur.execute("SELECT COUNT(*) FROM nodes;")
print(cur.fetchall())

[(575548,)]


In [3]:
# 3. Forming table for ways based on ways.csv file

cur.execute('DROP TABLE IF EXISTS ways')
conn.commit()

cur.execute('''
    Create Table ways(id INTEGER, user TEXT, uid INTEGER, version TEXT, changeset INTEGER, timestamp TEXT)
''')

conn.commit()


with open('ways.csv','r') as file:
    table = csv.DictReader(file)
    to_table = [(i['id'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in table]
    cur.executemany("INSERT INTO ways(id, user, uid, version, changeset, timestamp) VALUES (?,?,?,?,?,?);", to_table)

conn.commit()

In [4]:
# 4. Getting number of ways in the table:

cur.execute("SELECT COUNT(*) FROM ways;")
print(cur.fetchall())

[(68024,)]


In [5]:
# 5. Forming table for node_tags based on node_tags.csv file

cur.execute('DROP TABLE IF EXISTS nodes_tags')
conn.commit()

cur.execute('''
    Create Table nodes_tags(id INTEGER, key TEXT, value TEXT, type TEXT)
''')

conn.commit()

with open('nodes_tags.csv','r') as file:
    table = csv.DictReader(file)
    to_table = [(i['id'], i['key'], i['value'], i['type']) for i in table]
    cur.executemany("INSERT INTO nodes_tags(id, key, value, type) VALUES (?,?,?,?);", to_table)

conn.commit()

In [6]:
# 6. Forming table for way_tags based on way_tags.csv file

cur.execute('DROP TABLE IF EXISTS ways_tags')
conn.commit()

cur.execute('''
    Create Table ways_tags(id INTEGER, key TEXT, value TEXT, type TEXT)
''')

conn.commit()

with open('ways_tags.csv','r') as file:
    table = csv.DictReader(file)
    to_table = [(i['id'], i['key'], i['value'], i['type']) for i in table]
    cur.executemany("INSERT INTO ways_tags(id, key, value, type) VALUES (?,?,?,?);", to_table)

conn.commit()

In [7]:
# 6. Forming table for way_nodes based on way_nodes.csv file

cur.execute('DROP TABLE IF EXISTS ways_nodes')
conn.commit()

cur.execute('''
    Create Table ways_nodes(id INTEGER, node_id INTEGER, position INTEGER)
''')

conn.commit()

with open('ways_nodes.csv','r') as file:
    table = csv.DictReader(file)
    to_table = [(i['id'], i['node_id'], i['position']) for i in table]
    cur.executemany("INSERT INTO ways_nodes(id, node_id, position) VALUES (?,?,?);", to_table)

conn.commit()

In [8]:
# Analysis 1: Total Number of Unique Users and 10 Most Common Ones:


cur.execute("SELECT COUNT(DISTINCT(e.uid)) \
             FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e;")

print(cur.fetchall())


cur.execute("SELECT e.user, COUNT(*) as num FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e \
             GROUP BY e.user \
             ORDER BY num DESC LIMIT 10;")

print(cur.fetchall())

[(692,)]
[('chadbunn', 244599), ('butlerm', 69490), ('mvexel', 50240), ('wrk3', 37352), ('mash84121', 31834), ('woodpeck_fixbot', 22952), ('nemmer', 16151), ('balcoath', 13526), ('TheDutchMan13', 11184), ('Val', 9913)]


In [9]:
# Analysis 2: Distribution of Religion in the City:

import pprint
cur.execute ("SELECT tags.value, COUNT(*) as num FROM (SELECT * FROM nodes_tags UNION ALL \
             SELECT * FROM ways_tags) tags \
             WHERE tags.key LIKE '%religion'\
             GROUP BY tags.value \
             ORDER BY num DESC;")

pprint.pprint(cur.fetchall())

[('christian', 532),
 ('muslim', 3),
 ('buddhist', 2),
 ('jewish', 1),
 ('scientologist', 1),
 ('unitarian_universalist', 1)]


In [10]:
# Analysis 3: Number of drinking bars in the City:

cur.execute("SELECT COUNT(*) FROM nodes_tags WHERE value LIKE '%bar';")
print(cur.fetchall())

[(45,)]


In [11]:
# Analysis 4: Overview of Amenities Available in the City:

import pprint
cur.execute ("SELECT tags.value, COUNT(*) as num FROM (SELECT * FROM nodes_tags UNION ALL \
             SELECT * FROM ways_tags) tags \
             WHERE tags.key LIKE '%amenity'\
             GROUP BY tags.value \
             ORDER BY num DESC;")

pprint.pprint(cur.fetchall())

[('parking', 1020),
 ('restaurant', 709),
 ('place_of_worship', 540),
 ('fast_food', 294),
 ('school', 283),
 ('fuel', 141),
 ('bank', 124),
 ('cafe', 102),
 ('bench', 99),
 ('toilets', 68),
 ('drinking_water', 50),
 ('bar', 37),
 ('shelter', 36),
 ('bicycle_parking', 34),
 ('bicycle_rental', 33),
 ('parking_entrance', 28),
 ('library', 27),
 ('post_office', 27),
 ('fire_station', 26),
 ('pharmacy', 24),
 ('theatre', 22),
 ('atm', 20),
 ('car_wash', 20),
 ('fountain', 19),
 ('hospital', 18),
 ('cinema', 16),
 ('pub', 16),
 ('post_box', 15),
 ('vending_machine', 13),
 ('clinic', 11),
 ('doctors', 10),
 ('grave_yard', 10),
 ('public_building', 10),
 ('university', 10),
 ('dentist', 9),
 ('police', 9),
 ('waste_basket', 9),
 ('bicycle_repair_station', 8),
 ('car_rental', 8),
 ('recycling', 8),
 ('college', 7),
 ('nightclub', 7),
 ('veterinary', 7),
 ('bbq', 6),
 ('charging_station', 6),
 ('studio', 6),
 ('townhall', 6),
 ('arts_centre', 5),
 ('community_centre', 5),
 ('ice_cream', 5),
 ('

In [12]:
# Analysis 5:  25 Most Common Shops in the City:

import pprint
cur.execute ("SELECT value, COUNT(*) as num FROM nodes_tags \
             WHERE key = 'shop'\
             GROUP BY value \
             ORDER BY num DESC LIMIT 25;")

pprint.pprint(cur.fetchall())


[('clothes', 118),
 ('convenience', 66),
 ('yes', 48),
 ('supermarket', 46),
 ('hairdresser', 44),
 ('car_repair', 39),
 ('department_store', 36),
 ('beauty', 35),
 ('mobile_phone', 31),
 ('car', 27),
 ('jewelry', 24),
 ('shoes', 23),
 ('bicycle', 20),
 ('books', 20),
 ('bakery', 16),
 ('furniture', 15),
 ('deli', 13),
 ('dry_cleaning', 12),
 ('gift', 12),
 ('electronics', 11),
 ('copyshop', 10),
 ('variety_store', 10),
 ('sports', 9),
 ('car_parts', 8),
 ('tyres', 8)]


In [13]:
# Dataset Problem 1: Phone Numbers

import pprint
cur.execute ("SELECT value  FROM ways_tags \
             WHERE key = 'phone';")

pprint.pprint(cur.fetchall())

[('801-396-9625',),
 ('435-527-5585',),
 ('(801) 447-6860',),
 ('801.575.2345',),
 ('(801) 262-4653',),
 ('801-466-8751',),
 ('801-483-5420',),
 ('801-250-6396',),
 ('(801) 238-7300',),
 ('801.533.4527',),
 ('801-535-6110',),
 ('801-539-0852',),
 ('801-524-8200',),
 ('+1 801 524 8100',),
 ('+1-801-532-5501',),
 ('(801) 973-6271',),
 ('385-468-1500',),
 ('801.583.9513',),
 ('801-581-6326',),
 ('+1 801 662 1000',),
 ('+1 801 587 7000',),
 ('+1 801 536 3500',),
 ('801-298-6040',),
 ('(801) 966-4653',),
 ('801) 266-8621',),
 ('385-468-1400',),
 ('801-414-4103',),
 ('+1 801 964 3100',),
 ('+1 801 328 3288',),
 ('+1-801-487-7736',),
 ('+1-385-468-1305',),
 ('1-801-539-8888',),
 ('(801) 484-7651',),
 ('385-468-1440',),
 ('(801) 264-3800',),
 ('+1 801 314 4100',),
 ('+1 801 363 1037',),
 ('+1 801 739 6585',),
 ('+1 801 333 3175',),
 ('+1 801 428 0366',),
 ('(801) 595-0000',),
 ('8014855427',),
 ('+1 801 461 4248',),
 ('8017465200',),
 ('+1 801 359 4020',),
 ('801.483.5418',),
 ('801.330.4406',

In [15]:
# Displaying sizes of all project files:

import os

print "OSM File Size is:", os.path.getsize('SLC.osm')*1e-6, "MB" 
print "project.db file is:", os.path.getsize('project.db')*1e-6, "MB"
print "nodes.csv file is:", os.path.getsize('nodes.csv')*1e-6, "MB"
print "nodes_tags.csv file is:", os.path.getsize('nodes_tags.csv')*1e-6, "MB"
print "ways.csv file is:", os.path.getsize('ways.csv')*1e-6, "MB"
print "ways_nodes.csv file is:", os.path.getsize('ways_nodes.csv')*1e-6, "MB"
print "ways_tags.csv file is:", os.path.getsize('ways_tags.csv')*1e-6, "MB"


OSM File Size is: 127.842843 MB
project.db file is: 72.327168 MB
nodes.csv file is: 48.703848 MB
nodes_tags.csv file is: 1.998708 MB
ways.csv file is: 4.128661 MB
ways_nodes.csv file is: 16.130337 MB
ways_tags.csv file is: 8.107374 MB
